In [1]:
from netCDF4 import Dataset, num2date
import numpy as np
import json

In [2]:
# import data
dataset = Dataset('model.nc')

In [3]:
# interrogate dimensions
print(dataset.dimensions.keys())

dict_keys(['Time', 'DateStrLen', 'west_east', 'south_north', 'bottom_top', 'bottom_top_stag', 'soil_layers_stag', 'west_east_stag', 'south_north_stag', 'seed_dim_stag'])


In [6]:
# interrogate variable structure
print(dataset.variables['U10'])

<class 'netCDF4._netCDF4.Variable'>
float32 U10(Time, south_north, west_east)
    FieldType: 104
    MemoryOrder: XY 
    description: U at 10 M
    units: m s-1
    stagger: 
    coordinates: XLONG XLAT XTIME
unlimited dimensions: Time
current shape = (34, 139, 129)
filling on, default _FillValue of 9.969209968386869e+36 used


In [7]:
# interrogate variables
# find the u and v wind data
print("Check variables:")
print(dataset.variables.keys())

Check variables:
dict_keys(['Times', 'XLAT', 'XLONG', 'LU_INDEX', 'ZNU', 'ZNW', 'ZS', 'DZS', 'VAR_SSO', 'U', 'V', 'W', 'PH', 'PHB', 'T', 'THM', 'HFX_FORCE', 'LH_FORCE', 'TSK_FORCE', 'HFX_FORCE_TEND', 'LH_FORCE_TEND', 'TSK_FORCE_TEND', 'MU', 'MUB', 'NEST_POS', 'P', 'PB', 'FNM', 'FNP', 'RDNW', 'RDN', 'DNW', 'DN', 'CFN', 'CFN1', 'THIS_IS_AN_IDEAL_RUN', 'P_HYD', 'Q2', 'T2', 'TH2', 'PSFC', 'U10', 'V10', 'RDX', 'RDY', 'RESM', 'ZETATOP', 'CF1', 'CF2', 'CF3', 'ITIMESTEP', 'XTIME', 'QVAPOR', 'QCLOUD', 'QRAIN', 'QICE', 'QSNOW', 'QGRAUP', 'RE_CLOUD_GSFC', 'RE_RAIN_GSFC', 'RE_ICE_GSFC', 'RE_SNOW_GSFC', 'RE_GRAUPEL_GSFC', 'RE_HAIL_GSFC', 'QNICE', 'QNSNOW', 'QNRAIN', 'QNGRAUPEL', 'SHDMAX', 'SHDMIN', 'SNOALB', 'TSLB', 'SMOIS', 'SH2O', 'SMCREL', 'SEAICE', 'XICEM', 'SFROFF', 'UDROFF', 'IVGTYP', 'ISLTYP', 'VEGFRA', 'GRDFLX', 'ACGRDFLX', 'ACSNOM', 'SNOW', 'SNOWH', 'CANWAT', 'SSTSK', 'COSZEN', 'LAI', 'VAR', 'MAPFAC_M', 'MAPFAC_U', 'MAPFAC_V', 'MAPFAC_MX', 'MAPFAC_MY', 'MAPFAC_UX', 'MAPFAC_UY', 'MAPFAC_VX'

In [168]:
# USER input names for u and v wind variables
u_var = 'U10'
v_var = 'V10'

In [169]:
print("Check units:")
print(dataset.variables[u_var].units)

Check units:
m s-1


In [170]:
print("Check dimensions:")
print(dataset.variables[u_var].dimensions, dataset.variables[u_var].shape)

Check dimensions:
('Time', 'south_north', 'west_east') (34, 139, 129)


In [171]:
var_u = dataset.variables['U10'][0]
var_v = dataset.variables['V10'][0]
var_lats = dataset.variables['XLAT'][0]
var_longs = dataset.variables['XLONG'][0]

In [172]:
# Obtem os valores min e max da latitude
max_lat = var_lats[0][0]
min_lat = var_lats[len(var_lats[0])-1][0]
print('LAT min/max: {} / {}'.format(min_lat, max_lat))

LAT min/max: -16.076797485351562 / -34.680599212646484


In [173]:
# obtain max and min long values
max_long = var_longs[0][0]
min_long = var_longs[0][len(var_longs[0])-1]
print('LONG min/max: {} / {}'.format(min_long, max_long))

LONG min/max: -38.808189392089844 / -59.53180694580078


In [174]:
# os pontos superior-esquerdo (TL) e inferior-direito (BR) da grade de pontos
tl_lat = float(min_lat)
tl_long = float(max_long)
print('Top-Left: {} , {}'.format(tl_lat, tl_long))

Top-Left: -16.076797485351562 , -59.53180694580078


In [175]:
br_lat = float(max_lat)
br_long = float(min_long)
print('Bottom-Right: {} , {}'.format(br_lat, br_long))

Bottom-Right: -34.680599212646484 , -38.808189392089844


In [176]:
# set header variables for wind
nx = var_u.shape[0]
ny = var_u.shape[1]
dx = abs(max_lat - min_lat) / nx
dy = abs(max_long - min_long) / ny
tot = nx * ny
dx

0.13384030019636634

In [177]:
# get data for u wind
a = dataset.variables[u_var][:][0]
A = np.matrix(a)
b = A.flatten()
c = np.ravel(b).T
u_data = c.tolist()

In [178]:
# get data for v wind
a = dataset.variables[v_var][:][0]
A = np.matrix(a)
b = A.flatten()
c = np.ravel(b).T
v_data = c.tolist()

In [179]:
# format JSON
wind_data = [{
  "header": {
    "parameterNumberName": "eastward_wind",
    "parameterUnit": "m.s-1",
    "parameterNumber": 2,
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "numberPoints": tot,
    "dx": dx,
    "dy": dy,
    "la1": tl_lat,
    "lo1": tl_long,
    "la2": br_lat,
    "lo2": br_long,
    "refTime": "2017-02-01 23:00:00"
  },
  "data": u_data
}, {
  "header": {
    "parameterNumberName": "northward_wind",
    "parameterUnit": "m.s-1",
    "parameterNumber": 3,
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "numberPoints": tot,
    "dx": dx,
    "dy": dy,
    "la1": tl_lat,
    "lo1": tl_long,
    "la2": br_lat,
    "lo2": br_long,
    "refTime": "2017-02-01 23:00:00"
  },
  "data": v_data
}]

In [180]:
# write JSON for leaflet-velocity input
with open('wind.json', 'w') as outfile:  
    json.dump(wind_data, outfile, separators=(',', ':'))

In [45]:
# get data for temp from netCDF
temps = dataset.variables[temp_var][:][0]

KeyError: 'tsurf'

In [28]:
# get data for lat and lon
lats = dataset.variables['lat'][:]
lons = dataset.variables['lon'][:]

In [29]:
# loop through and create array
# temp is scaled from Kelvin to 0-1 with range 200K to 350K
# USER can edit display options
temp_data = [[0,0,0] for i in range(len(lats) * len(lons))]
for i in range(len(lats)):
    for j in range(len(lons)):
        temp_data[j + (i * len(lons))][0] = lats[i]
        temp_data[j + (i * len(lons))][1] = lons[j]
        temp_data[j + (i * len(lons))][2] = (temps[i,j] - 273.15) # + 273.15 for K
        #temp_data[j + (i * len(lons))][2] = str((temps[i,j] - 200)/150) if string is necessary

In [32]:
# apply non-overlapping moving window average to reduce data size by factor of 144 
# USER can edit grouping parameter
# number of points should not be more than several hundred for best performance
group = 12
lats_sm = lats.reshape(-1, group).mean(axis=1)
lons_sm = lons.reshape(-1, group).mean(axis=1)

In [33]:
# create new smaller temperature array 
temp_array = [[0] for i in range(len(lats) * len(lons))]
for i in range(len(temp_data)):
    temp_array[i] = temp_data[i][2]
temps_sm = np.array(temp_array).reshape(-1, group * group).mean(axis=1)

In [35]:
# reformat array to [lat, lon, temp]
temp_data_sm = [[0,0,0] for i in range(len(lats_sm) * len(lons_sm))]
for i in range(len(lats_sm)):
    for j in range(len(lons_sm)):
        temp_data_sm[j + (i * len(lons_sm))][0] = lats_sm[i]
        temp_data_sm[j + (i * len(lons_sm))][1] = lons_sm[j] -180
        temp_data_sm[j + (i * len(lons_sm))][2] = temps_sm[j + (i * len(lons_sm))] + 40

In [36]:
# write Javascript file for Leaflet.idw input
with open('temps_sm.js', 'w') as filehandle:  
    filehandle.write('var addressPoints = ' + str(temp_data_sm))